In [1]:
import pandas as pd
import numpy as np
import re

# Removing the tweets

In [2]:
def removing_tweets(data):
    """This function removes the tweets from the data"""
    indexes = []
    raw_commentaries = data.iloc[:data.shape[0] - 7,0]

    df = pd.DataFrame(raw_commentaries)

    df.dropna(inplace=True)
    
    for i in range(raw_commentaries.shape[0]):
        if ((': "' not in raw_commentaries[i]) and (' "' not in raw_commentaries[i])):
            indexes.append(i)
    df = df.iloc[indexes]

    df.reset_index(inplace=True, drop=True)
    return df


In [3]:
def index_no(df):
    """This function returns the index of 'Innings Start' string the data"""
    idx = list(df[df['Data'] == 'Innings Start'].index)

    return idx

# First Innings Data Processing

In [4]:
def flip_df(dataFrame):
    """This function flip the dataframe"""
    dataFrame = dataFrame.reindex(index=dataFrame.index[::-1])
    dataFrame.reset_index(inplace = True, drop=True)
    return dataFrame


def first_innings_commentary(data):
    """This function cleans first innings commentary """
    df_1 = removing_tweets(data)
    idx = index_no(df_1)
    first_innings = df_1.iloc[:idx[0],:]
    first_innings_1 = flip_df(first_innings)
    commentary_start_idx = None
    for commentary_start_idx in range(first_innings_1.shape[0]):
        if ("0.1" in first_innings_1.iloc[commentary_start_idx,0]):
            break
    beginneing_commentary = pd.DataFrame(first_innings_1.iloc[:commentary_start_idx, 0])

    ### Who won the toss
    toss_won = None
    for i in range(beginneing_commentary.shape[0]):
        if (("won" in beginneing_commentary.iloc[i,0]) and ("toss" in beginneing_commentary.iloc[i,0])) or (("toss" in beginneing_commentary.iloc[i,0])  and ("win" in beginneing_commentary.iloc[i,0])):
            toss_won =pd.DataFrame( {'Data':[beginneing_commentary.iloc[i,0]]})

    inning_1 = pd.DataFrame(first_innings_1.iloc[commentary_start_idx:, 0])
    txt_commentary = []
    for i in range(inning_1.shape[0]):
        arrr = inning_1.iloc[i,0]
        if (len(arrr.split(";")) > 2) :

            txt_commentary.append(arrr.split(";")[2])
        else:
            txt_commentary.append(arrr)

    inning_1 = pd.DataFrame({'Data':txt_commentary})

    inning_1.reset_index(inplace=True, drop=True)
    if toss_won is not None:
        inning_1 = toss_won.append(inning_1)

    
    indexes = []
    for i in range(inning_1.shape[0]):
        if (len(inning_1.iloc[i,0]) > 15):
            indexes.append(i)

    inning_1 = inning_1.iloc[indexes]
    

    inning_1.reset_index(inplace=True, drop=True)
    return inning_1

# Second Innings Commentary

In [5]:

def second_innings_commentary(data):
        """This function cleans second innings commentary"""
        df_1 = removing_tweets(data)
        idx = index_no(df_1)
        second_innings = df_1.iloc[idx[0]+1:idx[1],:]

        second_innings_1 = flip_df(second_innings)

        indexes_of_interviews = []
        for i in range(second_innings_1.shape[0]):
            if (': ' not in second_innings_1.iloc[i, 0]):
                indexes_of_interviews.append(i)

        second_innings_1 = second_innings_1.iloc[indexes_of_interviews,0]

        second_innings_1 = pd.DataFrame(second_innings_1)

        second_innings_1.reset_index(inplace=True, drop=True)

        txt_commentary_2 = []
        for i in range(second_innings_1.shape[0]):
            arrr = second_innings_1.iloc[i,0]
            if (len(arrr.split(";")) > 2) :
                txt_commentary_2.append(arrr.split(";")[2])
            else:
                txt_commentary_2.append(arrr)
        second_innings_1 = pd.DataFrame({'Data':txt_commentary_2})
        indexes = []
        for i in range(second_innings_1.shape[0]):
            if (len(second_innings_1.iloc[i,0]) > 15):
                indexes.append(i)

        second_innings_1 = second_innings_1.iloc[indexes]

        second_innings_1.reset_index(inplace=True, drop=True)
        return second_innings_1

### Player of the match

In [6]:
def player_of_match(data):
    """This function gets player of the match information from data"""
    plyr_of_match = data.iloc[data.shape[0]-7:,0]

    plyr_of_match.reset_index(inplace=True, drop=True)

    plyr_of_match_str = plyr_of_match[5]
    match_status = plyr_of_match[6]
    print(plyr_of_match_str)
    print(match_status)

    plyr_of_match_df = pd.DataFrame( {'Data':[match_status,plyr_of_match_str]})
    return plyr_of_match_df

In [7]:
def final_commentary(data):
    """This function aggregates all the data"""
    plyr_of_match_df = player_of_match(data)
    first_innnings = first_innings_commentary(data)
    second_innings = second_innings_commentary(data)
    final_commentary = first_innnings.append(second_innings)
    final_commentary = final_commentary.append(plyr_of_match_df)
    final_commentary.reset_index(inplace=True,drop=True)

    return final_commentary

In [8]:
import os.path
data = None
list_of_comm = []
for i in range(1,696):
    if i in [78,79,616, 621,644,647,648,646, 675,645, 581,483,484,495,496,555,556,557,82,406,407,434,435,69,70,92,94,264,265,266,267,288,233,343,246,247,248,249,250,251,252,276,277,278,296,297,295,102,118,119,158,159,166,167,168,169,170,171,193,199,205,215,229,237,257,284,316,332,334,363,393,394,19, 20, 21, 22, 32, 32, 45, 46, 51, 55, 58, 80, 81, 83, 84, 85, 88, 89, 94, 95, 101, 108, 109, 110, 111, 129, 130, 138, 139, 140, 141, 142, 143, 150, 165, 176, 177, 178, 179, 188, 204, 210, 211, 221, 222, 223, 224, 256, 258, 268, 365, 366, 421, 422, 431, 455]:
        continue
    if os.path.isfile(os.path.join(os.getcwd(), 'Commentaries', 'Text_Comm', 'commentary_{0}.txt'.format(i)).replace('\\', '/')):
        data = pd.read_fwf(os.path.join(os.getcwd(), 'Commentaries', 'Text_Comm', 'commentary_{0}.txt'.format(i)).replace('\\', '/'))
        final_comm_1 = final_commentary(data)
        final_comm_1.to_csv(os.path.join(os.getcwd(), 'Commentaries', 'commentary_{0}.csv'.format(i)).replace('\\', '/'))
        
        print(i)

Player of the match is Shakib Al Hasan, BDESH
Bangladesh won by 60 runs
1
Player of the match is Mitchell Swepson, AUS
Australia won by 3 wickets (with 6 balls remaining)
2
Player of the match is Mahmudullah, BDESH
Bangladesh won by 10 runs
3
Player of the match is Afif Hossain, BDESH
Bangladesh won by 5 wickets (with 8 balls remaining)
4
Player of the match is Nasum Ahmed, BDESH
Bangladesh won by 23 runs
5
Player of the match is Ashton Agar, AUS
Australia won by 6 wickets (with 117 balls remaining)
6
Player of the match is Nicholas Pooran, WI
West Indies won by 4 wickets (with 72 balls remaining)
7
Player of the match is Mitchell Starc, AUS
Australia won by 133 runs (D/L method)
8
Player of the match is Evin Lewis, WI
West Indies won by 16 runs
9
Player of the match is Mitchell Marsh, AUS
Australia won by 4 runs
10
Player of the match is Chris Gayle, WI
West Indies won by 6 wickets (with 31 balls remaining)
11
Player of the match is Shimron Hetmyer, WI
West Indies won by 56 runs
12
Pl

145
Player of the match is David Warner, AUS
Australia won by 86 runs
146
Player of the match is Steven Smith, AUS
Australia won by 7 wickets (with 30 balls remaining)
147
Player of the match is Mohammad Hafeez, PAK
Pakistan won by 6 wickets (with 14 balls remaining)
148
Player of the match is Matthew Wade, AUS
Australia won by 92 runs
149
Player of the match is Adam Zampa, AUS
Australia won by 41 runs
152
Player of the match is Asela Gunaratne, SL
Sri Lanka won by 2 wickets
153
Player of the match is Asela Gunaratne, SL
Sri Lanka won by 5 wickets
154
Player of the match is Trent Boult, NZ
New Zealand won by 24 runs
156
Player of the match is Ben Stokes, ENG
England won by 40 runs (D/L method)
157
Player of the match is David Warner, AUS
Australia won by 21 runs
160
Player of the match is Hardik Pandya, INDIA
India won by 5 wickets (with 13 balls remaining)
161
Player of the match is Virat Kohli, INDIA
India won by 50 runs
162
Player of the match is Hardik Pandya, INDIA
India won by 26

Player of the match is Steven Smith, AUS
Australia won by 73 runs
304
Player of the match is Morne Morkel, SA
South Africa won by 3 wickets (with 134 balls remaining)
305
Player of the match is Nathan Coulter-Nile, AUS
Australia won by 32 runs
306
Player of the match is Cameron White, AUS
Australia won by 2 wickets (with 1 ball remaining)
307
Player of the match is Cameron Boyce, AUS
Australia won by 7 wickets (with 44 balls remaining)
308
Player of the match is Rilee Rossouw, SA
South Africa won by 7 wickets (with 6 balls remaining)
309
Jared: "Brad Haddin sledging Kohli by shouting "It's all about you!" at him on TV. The irony is palpable."
Innings Start
310
2.05pm Warner and Rogers are out in the middle. Ashwin gives Warner a pat on the back, and Warner pats Ashwin back on his back. Lots of backs in there. Who's coming in with the new ball, to bend his back? It's Ishant.
Innings Start
311
Shri: "As India gets to 100, can't help but think that throughout this game, India has lost one

Player of the match is Shane Watson, AUS
Australia won by 17 runs (D/L method)
404
Player of the match is Shane Watson, AUS
Australia won by 7 wickets (with 29 balls remaining)
405
Player of the match is David Warner, AUS
Australia won by 94 runs
408
Player of the match is Saeed Ajmal, PAK
Match tied (Pakistan won the one-over eliminator)
409
Player of the match is Mohammad Hafeez, PAK
Pakistan won by 7 wickets (with 31 balls remaining)
410
Player of the match is Michael Hussey, AUS
Australia won by 3 wickets (with 18 balls remaining)
411
Player of the match is Chris Gayle, WI
West Indies won by 74 runs
417
Player of the match is Raza Hasan, PAK
Pakistan won by 32 runs
418
Steyn stops short at the delivery stride
Innings Start
419
Siddle has a short leg in place now
Innings Start
420
Shaun: "@notascientist, what era are you talking about? Remember that Lillee had his back put back together in the 1970s. Reid & Foster were 1980s. Are you talking before these periods when there was a les

Hilfenhaus is changing is line by coming around the wicket so we just wait for the sightscreen to be moved
Innings Start
511
Player of the match is Mohammad Amir, PAK
Pakistan won by 11 runs
513
Player of the match is Umar Akmal, PAK
Pakistan won by 23 runs
514
Player of the match is Shaun Tait, AUS
Australia won by 42 runs
515
Player of the match is Suraj Randiv, SL
Sri Lanka won by 7 wickets (with 21 balls remaining)
516
Player of the match is Virat Kohli, INDIA
India won by 5 wickets (with 7 balls remaining)
518
Player of the match is Jonathan Trott, ENG
England won by an innings and 157 runs
519
Ben Hilfenhaus will start things off to Andrew Strauss. Three slips and a gully wait...
Innings Start
520
Player of the match is Kevin Pietersen, ENG
England won by an innings and 71 runs
521
Player of the match is Hashim Amla, SA
South Africa won by 39 runs
522
Player of the match is AB de Villiers, SA
South Africa won by 8 wickets (with 71 balls remaining)
523
Player of the match is Albie

650
Player of the match is Mitchell Johnson, AUS
Australia won by 9 wickets (with 145 balls remaining)
652
Player of the match is MS Dhoni, INDIA
India won by 8 runs
653
Player of the match is Mitchell Swepson, AUS
Australia won by 3 wickets (with 6 balls remaining)
654
Player of the match is Brad Haddin, AUS
Australia won by 84 runs
655
a slip and a gully
Innings Start
656
Player of the match is Brett Lee, AUS
Australia won by an innings and 40 runs
657
Player of the match is Andrew Symonds, AUS
Australia won by 54 runs
662
Player of the match is Ricky Ponting, AUS
Australia won by 8 runs
663
Player of the match is Mitchell Johnson, AUS
Australia won by 9 wickets (with 153 balls remaining)
664
Player of the match is Michael Clarke, AUS
Australia won by 2 wickets (with 8 balls remaining)
665
Player of the match is Michael Hussey, AUS
Australia won by 4 wickets (with 68 balls remaining)
666
Player of the match is Andrew Symonds, AUS
Australia won by 105 runs
667
Player of the match is A